# Perform forward kinematics with a trajectory

In this notebook, we are going to generate a short video for a random robot movement you can watch in your browser.

First of all, we need some modules. You can create some simple modules built from geometric primitives yourself - there is a distinct tutorial to that purpose. If you don't want to, we can use the set of simple modules provided with the timor source code:

In [2]:
from pathlib import Path

# provide your own filepaths if you already did the module generation tutorial:
your_modules_json = Path('')
your_assets_directory = Path('')

if your_modules_json == Path(''):
    from timor.utilities.file_locations import get_module_db_files
    modules_file = get_module_db_files('geometric_primitive_modules')

2025-02-11 14:42:29,658 Timor INFO Loading custom configurations from /home/llama/.config/timor.config
2025-02-11 14:42:31,270 Timor INFO Getting robot modrob-gen2.
2025-02-11 14:42:31,273 Timor WARNING Could not fetch robot modrob-gen2 due to [Errno 17] File exists: '/home/llama/miniconda3/envs/timor/lib/python3.10/site-packages/cache/robots/modrob-gen2'.


Imports here:

In [3]:
import numpy as np
import timor
from timor.Module import *
from timor.utilities.visualization import animation
import matplotlib.pyplot as plt
import itertools
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm


#create the DB object
db = ModulesDB.from_json_file(modules_file)

#print out available modules in DB
print(db.by_name)
print(db.by_id)
# print(db.all_joints)
# print(db.all_connectors)


{'Revolute Joint': <timor.Module.AtomicModule object at 0x7fc1521859c0>, 'I shaped link 0.08-0.08-15': <timor.Module.AtomicModule object at 0x7fc1521869e0>, 'L shaped link 0.1-0.08-30': <timor.Module.AtomicModule object at 0x7fc1521f8c70>, 'Base': <timor.Module.AtomicModule object at 0x7fc152187430>, 'L shaped link 0.1-0.08-15': <timor.Module.AtomicModule object at 0x7fc1521f8460>, 'I shaped link 0.08-0.08-30': <timor.Module.AtomicModule object at 0x7fc152187c40>, 'Demo EEF': <timor.Module.AtomicModule object at 0x7fc152187d60>, 'L shaped link 0.1-0.08-45': <timor.Module.AtomicModule object at 0x7fc1521f9480>, 'I shaped link 0.08-0.08-45': <timor.Module.AtomicModule object at 0x7fc152186290>, 'Prismatic Joint': <timor.Module.AtomicModule object at 0x7fc1531e4190>}
{'J2': <timor.Module.AtomicModule object at 0x7fc1521859c0>, 'i_15': <timor.Module.AtomicModule object at 0x7fc1521869e0>, 'l_30': <timor.Module.AtomicModule object at 0x7fc1521f8c70>, 'base': <timor.Module.AtomicModule objec

In [4]:
#contains IDs we care about. The order is arbitrary, but if we want to make a robot out of this (i.e. kinematic tree) we should put in order.
modules = ('base', 'J2', 'i_45', 'J2', 'J2', 'eef')
A = ModuleAssembly.from_serial_modules(db, modules)
robot = A.to_pin_robot() #convert to pinocchio robot

viz = robot.visualize()
viz.viewer.jupyter_cell(height=400) # generate a jupyter cell to visualize the robot

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7032/static/


In [5]:
#arbitrary new joint state
q2 = np.array([np.pi, -np.pi/2, np.pi/2])


"""Forwards Kinematics (FK) and Forwards Dynamics (FD) methods belong to PinRobot object"""
  # FK https://timor-python.readthedocs.io/en/latest/autoapi/timor/Robot/index.html#timor.Robot.PinRobot.fk
  # FD https://timor-python.readthedocs.io/en/latest/autoapi/timor/Robot/index.html#timor.Robot.PinRobot.fd

# FK calculates the relative rigid body transform from the reference config at a given joint angle position. 
# You can tell it to update the collision hitbox and visual mesh if wanted. I don't think this robot can collide with itself.
print(f"Robot end effector coordinate transform at joint state {q2}", robot.fk(configuration = q2, collision = True, visual = True)) #default kind is tcp - tool center point
print(f"Robot joint coordinate transfom at joint state {q2}", robot.fk(configuration = q2, kind = 'joints', collision = True, visual = True))
print(f"Robot full frames coordinate transfom at joint state {q2}", robot.fk(configuration = q2, kind = 'full', collision = True, visual = True))


#visual just updates the state of the visual geometries, we have to actually visualize to see it
vis = robot.visualize()

#Current robot velocities
print(f"Current robot joint positions (configuration): {robot.configuration}")
print(f"Current joints: {robot.joints}")
print(f"Current joints parents: {robot.parents}")

import pinocchio as pin
print(pin.Force.Random())


print(f"Current robot joint velocities: {robot.velocities}")

# FD calculates joint accelerations (ddq) given joint config (q) and joint velocity (dq) 
print("FD:", robot.fd(tau = [1, 0, 0], motor_inertia = False, friction = True))




Robot end effector coordinate transform at joint state [ 3.14159265 -1.57079633  1.57079633] [[-0.    1.   -0.    0.5 ]
 [-0.   -0.   -1.   -0.19]
 [-1.   -0.    0.    0.8 ]
 [ 0.    0.    0.    1.  ]]
Robot joint coordinate transfom at joint state [ 3.14159265 -1.57079633  1.57079633] ([[ 1.    0.    0.    0.2 ]
 [-0.    1.    0.   -0.  ]
 [ 0.   -0.    1.    0.05]
 [ 0.    0.    0.    1.  ]], [[-0.   0.   1.   0.2]
 [-1.  -0.  -0.   0. ]
 [ 0.  -1.   0.   0.8]
 [ 0.   0.   0.   1. ]], [[-0.   1.  -0.   0.5]
 [-0.  -0.  -1.   0. ]
 [-1.  -0.   0.   0.8]
 [ 0.   0.   0.   1. ]])
Robot full frames coordinate transfom at joint state [ 3.14159265 -1.57079633  1.57079633] ([[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]], [[ 1.  0.  0.  0.]
 [ 0. -1. -0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0.  0.  1.]], [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]], [[-0.   -0.    1.    0.05]
 [ 0.   -1.    0.    0.  ]
 [ 1.    0.    0.    0.05]
 [ 0.    0.    0.    1.  ]], [[-0.    0

In [6]:
## attempting to calculate joint torques:

def check_feasibility_with_limits(computed_torques: np.ndarray, torque_limits: np.ndarray) -> bool:
    """
    Checks if the computed torques are feasible given the upper and lower allowable torque limits for each joint.

    Parameters:
        computed_torques (numpy.ndarray): The torques computed by the static_torque function (n_joints x 1).
        torque_limits (numpy.ndarray): A 2D array containing the lower and upper limits for each joint's torque.
                                       Shape: (n_joints, 2) where each row contains [lower_limit, upper_limit].

    Returns:
        bool: True if all computed torques are within the allowable limits, False otherwise.
    """
    # Check if the computed torques are within the lower and upper limits for each joint
    lower_limits = torque_limits[:, 0]
    upper_limits = torque_limits[:, 1]

    # Check if all computed torques are within their respective limits
    feasible = np.all((computed_torques >= lower_limits) & (computed_torques <= upper_limits))

    
    if feasible:
        print("All computed torques are within the allowable limits.")
    else:
        # Identify the joints where the torque exceeds the limits
        exceeding_joints_lower = np.where(computed_torques < lower_limits)[0]
        exceeding_joints_upper = np.where(computed_torques > upper_limits)[0]
        
        # Print which joints exceed the torque limits
        if len(exceeding_joints_lower) > 0:
            print(f"Joints {exceeding_joints_lower} have computed torques below the lower limit.")
        if len(exceeding_joints_upper) > 0:
            print(f"Joints {exceeding_joints_upper} have computed torques above the upper limit.")
    
    return feasible


print("static: ", robot.static_torque(robot.q, f_ext=None))

# no external end effector force
eef_wrench = np.array([0, 0, 0, 0, 0, 0])  # A downward force of 10 units in the Z direction
joint_torques = robot.id(q=None, dq=None, ddq=None, motor_inertia=True, friction=True, eef_wrench=eef_wrench)
# print("joint_torques:", joint_torques)

# adding a downward end effector force
eef_wrench = np.array([-0, -0, -1000, 0, 0, 0])  # A downward force of 10 units in the Z direction
proposed_joint_torques = robot.id(q=None, dq=None, ddq=None, motor_inertia=True, friction=True, eef_wrench=eef_wrench)
print("joint_torques:", proposed_joint_torques, type(proposed_joint_torques))

# Maximum allowable torques for each joint
torque_limits = np.array([[-5.0e+06, 5.0e+06],
                          [-2.0e+06, 2.0e+06],
                          [-1.0e+10, 1.0e+10]])

# Check feasibility
feasible = check_feasibility_with_limits(proposed_joint_torques, torque_limits)
print("feasible? :", feasible==True)

static:  [-2.85332688e-16 -2.91239776e-01  0.00000000e+00]
joint_torques: [-3.70247367e-14 -2.91239776e-01  0.00000000e+00] <class 'numpy.ndarray'>
All computed torques are within the allowable limits.
feasible? : True
